Perform logistic regression and analysis with XGboost

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import math 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
Charter_Schools_Analysis = pd.read_pickle("Data/AnalysisSet/Charter_Schools_Analysis.pkl")    

Charter_Schools_Analysis2 = Charter_Schools_Analysis[['School_x', 'ANN_TOTAL_ENROLL', 'Pct_White', 'Pct_Male','Pct_ELL', 'Pct_FRPM','year' ,'open_two_years', 'openyear', 'percent_unemployed', 'percent_nohs', 'SOCType', 'County', 
                                                     'Virtual', 'YearRoundYN','Magnet', 'PS_score']]

Charter_Schools_Analysis2['First_School'] = (Charter_Schools_Analysis2.groupby('School_x').cumcount() == 0).astype(int)

Charter_Schools_Analysis2 = Charter_Schools_Analysis2.loc[Charter_Schools_Analysis2['First_School'] > 0]
Charter_Schools_Analysis2 = Charter_Schools_Analysis2.loc[Charter_Schools_Analysis2['openyear'] > 2003]

Charter_Schools_Analysis2['virtual_school'] = np.where(Charter_Schools_Analysis2['Virtual'] == "P", 0, 1) 
Charter_Schools_Analysis2['Year_Round_school'] = np.where(Charter_Schools_Analysis2['YearRoundYN'] == "N", 0, 1)
Charter_Schools_Analysis2['Magnet_school'] = np.where(Charter_Schools_Analysis2['Magnet'] == "N", 0, 1)

Charter_Schools_Analysis2 = Charter_Schools_Analysis2.dropna()
Charter_Schools_Analysis2 = Charter_Schools_Analysis2.reset_index(drop=True)

Charter_Schools_Analysis3 = Charter_Schools_Analysis2.drop(columns=['Virtual', 'YearRoundYN', 'Magnet'])

In [ ]:
Charter_Schools_Pre_2017 = Charter_Schools_Analysis3.loc[Charter_Schools_Analysis3['year'] < 17]

X_Pre_2017 = Charter_Schools_Pre_2017.drop(columns=['School_x', 'open_two_years', 'First_School', 'SOCType', 'County'])                                                                   
y_Pre_2017 = Charter_Schools_Pre_2017['open_two_years']
X_train, X_test, y_train, y_test = train_test_split(X_Pre_2017, y_Pre_2017, test_size=0.4, random_state=42)

Basic Logistic Regression

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [ ]:
list(X_train.columns)

In [ ]:
results = sm.Logit(y_train, X_train).fit()
print(results.summary())

In [ ]:
predicted_test_logit = results.predict(X_test)

In [ ]:
predicted_test_logit = pd.DataFrame(predicted_test_logit)
predicted_test_logit = predicted_test_logit.rename(columns={0: "Score"})

In [ ]:
from sklearn import metrics

predicted_test_list_logit = predicted_test_logit.values.tolist()

In [ ]:
y_test_np = np.array(y_test)

In [ ]:
predicted_test_np = np.array(predicted_test_list_logit)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_np, predicted_test_np)

roc_auc = metrics.auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Run analysis using XGboost

In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics  
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import log_loss

In [ ]:
mean_train = np.mean(y_train)

In [ ]:
baseline_predictions = np.ones(y_test.shape) * mean_train

In [ ]:
baseline_predictions_classified = np.ones(y_test.shape)

In [ ]:
accuracy_baseline = log_loss(y_test, baseline_predictions_classified)
print("Baseline accuracy is {:.2f}".format(accuracy_baseline))

Tune hyper-parameters using a sequential grid search.

In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'binary:logistic',
}

In [ ]:
params['eval_metric'] = "logloss"

num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'logloss'},
    early_stopping_rounds=10
)
cv_results

In [ ]:
cv_results['test-logloss-mean'].min()

In [ ]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(1,12)
    for min_child_weight in range(1,8)
]

In [ ]:
# Define initial best params and MAE
min_error = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )
    # Update best error
    mean_error = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print("\tError {} for {} rounds".format(mean_error, boost_rounds))
    if mean_error < min_error:
        min_error = mean_error
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, Error: {}".format(best_params[0], best_params[1], min_error))

In [ ]:
params['max_depth'] = 3
params['min_child_weight'] = 1

In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1,11)]
    for colsample in [i/10. for i in range(1,11)]
]

In [ ]:
min_error = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_error = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print("\tError {} for {} rounds".format(mean_error, boost_rounds))
    if mean_error < min_error:
        min_error = mean_error
        best_params = (subsample,colsample)
print("Best params: {}, {}, Error: {}".format(best_params[0], best_params[1], min_error))

In [ ]:
params['subsample'] = 1.0
params['colsample_bytree'] = 0.6

In [ ]:
%time
# This can take some time…
min_error = float("Inf")
best_params = None
for eta in [.9, .8, .7, .6, .5, .4, .3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time cv_results = xgb.cv(params, dtrain, num_boost_round=num_boost_round, seed=42, nfold=5, metrics=['logloss'],early_stopping_rounds=10)
    # Update best score
    mean_error = cv_results['test-logloss-mean'].min()
    boost_rounds = cv_results['test-logloss-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_error, boost_rounds))
    if mean_error < min_error:
        min_error = mean_error
        best_params = eta
print("Best params: {}, Error: {}".format(best_params, min_error))

In [ ]:
params['eta'] = .3

In [ ]:
params

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

In [ ]:
best_model.save_model("my_model.model")

In [ ]:
y_test_np = np.array(y_test)

In [ ]:
x_np = np.array(best_model.predict(dtest))

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_np, x_np)

roc_auc = metrics.auc(fpr, tpr)

In [ ]:
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_xgb = np.where(x_np > .8, 1, 0) 

confusion_matrix(y_test_np, y_pred_xgb)

In [ ]:
y_pred_logit = np.where(predicted_test_logit > .8, 1,0)

confusion_matrix(y_test_np, y_pred_logit)

In [ ]:
Charter_Schools_Post_2016 = Charter_Schools_Analysis3.loc[Charter_Schools_Analysis3['year'] > 16]

In [ ]:
X_Post_2016 = Charter_Schools_Post_2016.drop(columns=['School_x', 'open_two_years', 'First_School', 'SOCType', 'County'])      

In [ ]:
X_Post_2016['const'] = 1

In [ ]:
X_Post_2016 = X_Post_2016[['const', 'ANN_TOTAL_ENROLL', 'Pct_White', 'Pct_Male', 'Pct_ELL', 'Pct_FRPM', 'year',
                           'openyear', 'percent_unemployed', 'percent_nohs', 'PS_score', 'virtual_school',
                           'Year_Round_school', 'Magnet_school']]

In [ ]:
d_predict = xgb.DMatrix(X_Post_2016)
Predicted_Post_2016 = best_model.predict(d_predict)
Charter_Schools_Post_2016['predicted_survival'] = Predicted_Post_2016
Charter_Schools_Post_2016['Safe'] = np.where(Charter_Schools_Post_2016['predicted_survival'] > .8, "Safe", "Risky")
Charter_Schools_Post_2016_Out = Charter_Schools_Post_2016[['School_x', 'predicted_survival', 'Safe']]

In [ ]:
Charter_Schools_Post_2016_Out2 = Charter_Schools_Analysis[['School_x', 'ANN_TOTAL_ENROLL', 'year', 'Latitude_x', 'Longitude_x', 'CDS_CODE']]
Charter_Schools_Post_2016_Out2['First_School'] = (Charter_Schools_Post_2016_Out2.groupby('School_x').cumcount() == 0).astype(int)                                                                
Charter_Schools_Post_2016_Out2 = Charter_Schools_Post_2016_Out2.loc[Charter_Schools_Post_2016_Out2['First_School'] > 0]
Charter_Schools_Post_2016_Out2 = Charter_Schools_Post_2016_Out2.loc[Charter_Schools_Post_2016_Out2['year'] > 16]
Charter_Schools_Analysis_Out = Charter_Schools_Post_2016_Out.merge(Charter_Schools_Post_2016_Out2, left_on='School_x', right_on='School_x')
Charter_Schools_Analysis_Out = Charter_Schools_Analysis_Out.rename(columns={ "School_x" : "School", "Latitude_x" : "Latitude", "Longitude_x": "Longitude"})
Charter_Schools_Analysis_Out = Charter_Schools_Analysis_Out[['School', 'ANN_TOTAL_ENROLL', 'CDS_CODE', 'predicted_survival', 'Safe', 'Latitude', 'Longitude']]

In [ ]:
Charter_Schools_Analysis_Out.to_pickle("Data/AnalysisSet/Charter_Schools_Analysis_Out.pkl")

In [ ]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point, MultiPoint
from geopandas import GeoDataFrame

In [ ]:
Charter_Schools_Analysis = pd.read_pickle("Data/AnalysisSet/Charter_Schools_Analysis_Out.pkl")  

In [ ]:
Charter_Schools_Analysis['Latitude'] = Charter_Schools_Analysis['Latitude'].astype(float)
Charter_Schools_Analysis['Longitude'] = Charter_Schools_Analysis['Longitude'].astype(float)

In [ ]:
Charter_Schools_Analysis['First_Lat'] = (Charter_Schools_Analysis.groupby('Latitude').cumcount() == 0).astype(int)
Charter_Schools_Analysis = Charter_Schools_Analysis.loc[Charter_Schools_Analysis['First_Lat'] > 0]
Charter_Schools_Analysis = Charter_Schools_Analysis.reset_index(drop=True)

In [ ]:
from geopy.distance import geodesic

Charter_Schools_Analysis['location_home'] = list(zip(Charter_Schools_Analysis['Latitude'], Charter_Schools_Analysis['Longitude']))

Charter_Analysis_Neighbor = Charter_Schools_Analysis[['School', 'location_home', 'Latitude', 'Longitude', 'predicted_survival', 'Safe', 'ANN_TOTAL_ENROLL']]

for i in range(len(Charter_Analysis_Neighbor)):
    distance_array = []
    d_i = Charter_Analysis_Neighbor['location_home'][i]
    for j in range(len(Charter_Analysis_Neighbor)):
        d = (geodesic(d_i, Charter_Analysis_Neighbor['location_home'][j]).miles)
        distance_array.append(d)
        
    distance_df = pd.DataFrame(distance_array)
    distance_df = distance_df.rename(columns={0: "Distance"})
        
    Charter_Analysis_Neighbor2 =  Charter_Analysis_Neighbor.join(distance_df, how='outer')
    Charter_Analysis_Neighbor3 = Charter_Analysis_Neighbor2.nsmallest(6, 'Distance')
    Charter_Analysis_Neighbor4 = Charter_Analysis_Neighbor3[0:6] 
    Charter_Analysis_Neighbor4 = Charter_Analysis_Neighbor4.reset_index(drop=True)
        
    Charter_Analysis_Neighbor4['Indexi'] = Charter_Analysis_Neighbor4.index
    Charter_Analysis_Neighbor4['Indexj'] = i
    Charter_Analysis_Neighbor5 = Charter_Analysis_Neighbor4.pivot(index= "Indexj", columns='Indexi')
   
    if i == 0:
        Charter_Schools_Wide = Charter_Analysis_Neighbor5
    else:
        Charter_Schools_Wide = Charter_Schools_Wide.append(Charter_Analysis_Neighbor5, ignore_index=True)

In [ ]:
Charter_Schools_Wide.to_csv('Data/AnalysisSet/Charter_Analysis_Output.csv')